In [1]:
# set of (frozen) sets
sos = lambda *S: set(frozenset(A) for A in S)

# Probability Space

**<u>Definition</u>**

A probability space consists of three objects $(\Omega, \mathcal{F}, P)$:

* $\Omega$ &ndash; sample space: a nonempty set

* $\mathcal{F}$ &ndash; $\sigma$-algebra on $\Omega$: a set of sets

* $P$ &ndash; probability measure on $\mathcal{F}, \Omega$: a function mapping $P: \mathcal{F} \to [0, 1]$

## Sample Space $\Omega$

In [2]:
Ω = {1, 2, 3}

## Algebra $\mathcal{F}_{a}$

**<u>Definition</u>**

1. Sample space: $\Omega \in \mathcal{F}_{a}$ (and also empty set $\emptyset \in \mathcal{F}_a$).

2. Closed under complements: $A \in \Omega \implies A^c \in \mathcal{F}_{a}$.

3. Closed under finite union: $A, B \in \Omega \implies A \cup B \in \mathcal{F}_{a}$.

## $\sigma$-Algebra $\mathcal{F}$

**<u>Definition</u>**

Similar definition as that of an algebra, but closed under countable (infinite) union.

1. Sample space: $\Omega \in \mathcal{F}$ (and also empty set $\emptyset \in \mathcal{F}$).


2. Closed under complements: $A \in \Omega \implies A^c \in \mathcal{F}$.

3. Closed under countable (infinite) union: $\bigcup\limits_{i=1}^{\infty}A_i \in F, A_i \in \Omega.$

In [3]:
def algebra(generating_set, sample_space):

    F = [S for S in generating_set if S.issubset(sample_space)]
    
    if sample_space not in F:
        F.append(sample_space)

    if set() not in F:
        F.append(set())

    while True:
        
        F_old = F.copy()

        for A in F:
            # closure under complements
            A_comp = sample_space - A
            if A_comp not in F:
                F.append(A_comp)

            for B in F:
                # closure under finite union
                union = A.union(B)
                if union not in F:
                    F.append(union)

        if sos(*F_old)==sos(*F):
            return sos(*F)

In [4]:
algebra(sos({1}), Ω)

{frozenset(), frozenset({1}), frozenset({2, 3}), frozenset({1, 2, 3})}

## The Power Set

Given a set $S$, its power set $2^S$  is the set of all subsets of $S$. The power set of a sample space $\Omega$ is the largest $\sigma$-algebra on $\Omega$.

In [5]:
def power_set(S):
    
    ps = [{x} for x in S]
    
    for x in ps:
        for y in ps:
            union = x.union(y)
            if union not in ps:
                ps.append(union)
    
    # add empty set
    ps.append(set())
    
    return sos(*ps)

In [6]:
ps = power_set(Ω)
len(ps)

8

In [7]:
x = [0]

for y in x:
    x.append(y+1)
    if len(x) > 100:
        break
        
print(x)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


## Probability Measure $P$
A probability measure is a function $P: \Omega \to [0, 1]$ such that:

1. $P(\emptyset) = 0$
2. $P(\Omega) = 1$
3. For disjoint subsets $A_1, A_2, ... \in \mathcal{F}$: 
$$
P \left( \dot\bigcup_{i=1}^{\infty} A_i \right) = \sum_{i=1}^{\infty} P \left( A_i \right)
$$

In [14]:
import numpy as np
np.log(1) + np.log(2)

0.6931471805599453

In [30]:
# example of probability measure (for discrete prob space)
def P(A, Ω):
    return len(A) / len(Ω)

def lesbegue_measure(A, Ω):
    pass

In [150]:
class interval(object):
    
    def __init__(self, range_str):
        
        
        range_str = range_str.strip()
        
        lb, rb = range_str[0], range_str[-1]
        
        self.inf, self.sup = [float(x) for x in range_str[1:-1].split(',')]
        
        if self.inf == int(self.inf):
            self.inf = int(self.inf)
            
        if self.sup == int(self.sup):
            self.sup = int(self.sup)
            
        
        if lb == '(':
            self.lopen = True
        elif lb == '[':
            self.lopen = False
        else:
            raise Exception('Left bracket not specified')
            
        if rb == ')':
            self.ropen = True
        elif rb == ']':
            self.ropen = False
        else:
            raise Exception('Right bracket not specified')
    
    def __repr__(self):
        
        lb = '(' if self.lopen else '['
        rb = ')' if self.ropen else ']'
        
        return lb + f'{self.inf}, {self.sup}' + rb
    
    @property
    def length(self):
        return self.sup - self.inf
    
    @property
    def maximum(self):
        if self.ropen:
            return None
        else:
            return self.sup
    
    @property
    def minimum(self):
        if self.lopen:
            return None
        else:
            return self.inf
        
    def __lt__(self, other):
        
        if self.lopen:
            return self.inf <= other 
        else:
            return self.inf < other 
        
    def __gt__(self, other):
        
        if self.ropen:
            return self.sup >= other
        else:
            return  self.sup > other
    
    def __le__(self, other):
        return self.inf <= other
    
    def __ge__(self, other):
        return self.sup >= other

# Borel Set

* Sigma-algebra constructed with all open intervals

In [27]:
class Naturals(object):
    
    def __init__(self, start=0):
        self.start = start
        self._gen = self.generator(start)
        
    def __repr__(self):
        return 'ℕ'
        
    def __contains__(self, other):
            
        if int(other) == other:
            return other >= self.start
        else:
            return False
    
    def generator(self, n):
        yield n
        yield from self.generator(n+1)
        
    def next_n(self):
        return next(self._gen)
        
N = Naturals()
1 in N

True

# Random Variable


A random variable is a function $X : \Omega \to \mathbb{R}$ such that, given probability space $(\Omega, \mathcal{F}, P)$:

$$
\lbrace \omega \in \Omega : X(\omega) \le x \rbrace \in \mathcal{F} \quad \forall x \in \mathbb{R}.
$$

in other words, for all $x\in \mathbb{R}$, the inverse image of $X$ on interval $(-\infty, x]$ is in $F$:

$$
\lbrace X \leq x \rbrace = X^{-1}((-\infty, x]) \in \mathcal{F}.
$$

In [ ]:
import time

In [25]:
def X(ω):
    
    return 2* ω

def X_inv(X, x, Ω):
    
    res = set()
    for ω in Ω:
        if X(ω) <= x:
            res.add(ω)
    return res


def is_rv(X, Ω, F, t=1.0, return_xlim=False):
    '''
    Returns boolean if X is a r.v.
    
        Parameters:
            X (function):     A random variable candidate
            Ω (set):          The sample space
            F (set):          Sigma algebra on Ω (set of set)
            t (int or float): Time limit for testing (default is 1.0)
            
        Returns:
            is_rv (bool): Whether X is a r.v.
    '''
    
    t0, x= time.time(), 0
    while time.time()-t0 < 1:
        
        if X_inv(X, x, Ω) not in F or X_inv(X, -x, Ω) not in F:
            return False
        
        x+=1
    
    if return_xlim:
        return True, [-x, x]
    else:
        return True
    

In [ ]:
is_rv(X, Ω, ps)